# Jupyter Notebook to test Rayyan Loader

## Install dependencies

```bash
pip install -r notebook-requirements.txt
```


## Configure OpenAI with your API key

Make sure you have a file named `.env` in the same directory as this notebook, with the following contents:

```
OPENAI_API_KEY=<your key here>
OPENAI_ORGANIZATION=<your organization here>
```

The organization is optional, but if you are part of multiple organizations, you can specify which one you want to use. Otherwise, the default organization will be used.

Optionally, to enable NewRelic monitoring, add the following to your `.env` file:

```
NEW_RELIC_APP_NAME=<your app name here>
NEW_RELIC_LICENSE_KEY=<your key here>
```

In [ ]:
import os
import sys
import logging
from dotenv import load_dotenv

logging.basicConfig(stream=sys.stderr, level=logging.INFO)
logger = logging.getLogger(__name__)

load_dotenv()  # take environment variables from .env.

logger.debug(f"NewRelic application: {os.getenv('NEW_RELIC_APP_NAME')}")

## Load a Rayyan review into LLama Index

Make sure to have a Rayyan credentials file in `rayyan-creds.json`.
Check the [Rayyan SDK](https://github.com/rayyansys/rayyan-python-sdk) for more details.


In [ ]:
import os
from time import time

from nr_openai_observability import monitor
from llama_index import VectorStoreIndex, download_loader

if os.getenv("NEW_RELIC_APP_NAME") and os.getenv("NEW_RELIC_LICENSE_KEY"):
    monitor.initialization(application_name=os.getenv("NEW_RELIC_APP_NAME"))

# Uncomment to download the loader from another repository
# RayyanReader = download_loader("RayyanReader", loader_hub_url="https://raw.githubusercontent.com/rayyansys/llama-hub/rayyan-loader/llama_hub")
RayyanReader = download_loader("RayyanReader")
loader = RayyanReader(credentials_path="rayyan-creds.json")

# documents = loader.load_data(review_id=746345, filters={"search[value]": "outcome"})
documents = loader.load_data(review_id=746345)
logger.info("Indexing articles...")
t1 = time()
review_index = VectorStoreIndex.from_documents(documents)
t2 = time()
logger.info(f"Done indexing articles in {t2 - t1:.2f} seconds.")

INFO:root:Signed in successfully to Rayyan as: Hossam Hammady!
INFO:root:Working on review: 'PICO on-demand' with 900 total articles.
INFO:root:Fetching articles from Rayyan...
100%|██████████| 900/900 [00:05<00:00, 166.16it/s]
INFO:__main__:Indexing articles...
INFO:__main__:Done indexing articles in 42.46 seconds.


## Query LLama Index about the review data

In [ ]:
query_engine = review_index.as_query_engine()
prompts = [
    "What are the most used interventions?",
    "What is the most common population?",
    "Are there studies about children?",
    "Do we have any studies about COVID-19?",
    "Are there any multi-center randomized clinical trials?",
]
for idx, prompt in enumerate(prompts):
    print(f"❓ Query {idx + 1}/{len(prompts)}: {prompt}")
    print("Waiting for response...")
    response = query_engine.query(prompt)
    print(f"🤖 {response.response}")
    print("Relevant articles:")
    for article in response.metadata.values():
        print(f"- [{article['id']}] {article['title']}")
    print()

❓ Query 1/5: What are the most used interventions?
Waiting for response...
🤖 The most used interventions mentioned in the context are the Healthy Choices motivational interviewing intervention and the PlayForward: Elm City Stories videogame intervention. These interventions were developed to target multiple risk behaviors among HIV-positive youth and young minority teens, respectively. The Healthy Choices intervention focused on reducing alcohol and marijuana use, while the PlayForward intervention aimed to teach knowledge and skills for preventing HIV infection.
Relevant articles:
- [540581301] Alcohol and marijuana use outcomes in the healthy choices motivational interviewing intervention for HIV-positive youth
- [540581177] A videogame intervention for risk reduction and prevention in young minority teens

❓ Query 2/5: What is the most common population?
Waiting for response...
🤖 The most common population in the given context is fisher-folk communities (FFC) in Uganda.
Relevant art